In [3]:
import numpy as np 
import pandas as pd 
import tensorflow as tf

In [4]:
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd

In [5]:
import pandas as pd
metadata= pd.read_csv(r"C:\Users\Rashiqua Munshi\Downloads\ProjectSchool\UrbanSound8K.csv")

In [6]:
metadata.tail()

,slice_file_name,fsID,start,end,salience,fold,classID,class
8727,99812-1-2-0.wav,99812,159.522205,163.522205,2,7,1,car_horn
8728,99812-1-3-0.wav,99812,181.142431,183.284976,2,7,1,car_horn
8729,99812-1-4-0.wav,99812,242.691902,246.197885,2,7,1,car_horn
8730,99812-1-5-0.wav,99812,253.209850,255.741948,2,7,1,car_horn
8731,99812-1-6-0.wav,99812,332.289233,334.821332,2,7,1,car_horn


In [7]:
metadata["class"].value_counts()

class
dog_bark            1000
children_playing    1000
air_conditioner     1000
street_music        1000
engine_idling       1000
jackhammer          1000
drilling            1000
siren                929
car_horn             429
gun_shot             374
Name: count, dtype: int64

In [9]:
import os
audio_dataset_path=r"C:\Users\Rashiqua Munshi\Downloads\ProjectSchool\archive"

In [10]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file) 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features   

In [11]:
import numpy as np
from tqdm import tqdm
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

3549it [00:48, 71.81it/s] C:\Users\Rashiqua Munshi\Anaconda3\envs\tensorflow1\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
8315it [01:52, 86.63it/s] C:\Users\Rashiqua Munshi\Anaconda3\envs\tensorflow1\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
8328it [01:52, 97.55it/s]C:\Users\Rashiqua Munshi\Anaconda3\envs\tensorflow1\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(
8732it [01:58, 73.81it/s]


In [12]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-211.93698, 62.581203, -122.81315, -60.745293...",dog_bark
1,"[-417.0052, 99.336624, -42.995586, 51.073326, ...",children_playing
2,"[-452.39316, 112.36253, -37.578068, 43.195866,...",children_playing
3,"[-406.47922, 91.1966, -25.043556, 42.78452, 11...",children_playing
4,"[-439.63873, 103.86224, -42.658787, 50.690277,...",children_playing


In [13]:
X=np.array(extracted_features_df["feature"].tolist())
y=np.array(extracted_features_df["class"].tolist())

In [14]:
X.shape

(8732, 40)

In [15]:
y.shape

(8732,)

In [16]:
y=np.array(pd.get_dummies(y))

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [18]:
X_train.shape

(5937, 40)

In [19]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Activation,Dropout
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [20]:
model = Sequential()
model.add(Dense(100,input_shape=(40,)))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(200))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dropout(0.2))
          
model.add(Dense(10))
model.add(Activation("softmax"))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4100      
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 activation_1 (Activation)   (None, 200)               0         
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               2

In [22]:
model.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")

In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_valid, y_valid), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
165/186 [=========================>....] - ETA: 0s - loss: 5.3979 - accuracy: 0.1583
Epoch 1: val_loss improved from inf to 2.20187, saving model to saved_models\audio_classification.hdf5
186/186 [==============================] - 1s 3ms/step - loss: 5.0442 - accuracy: 0.1615 - val_loss: 2.2019 - val_accuracy: 0.1469
Epoch 2/100
183/186 [============================>.] - ETA: 0s - loss: 2.1740 - accuracy: 0.2037
Epoch 2: val_loss improved from 2.20187 to 2.04567, saving model to saved_models\audio_classification.hdf5
186/186 [==============================] - 0s 2ms/step - loss: 2.1741 - accuracy: 0.2043 - val_loss: 2.0457 - val_accuracy: 0.2844
Epoch 3/100
161/186 [========================>.....] - ETA: 0s - loss: 2.0215 - accuracy: 0.2962
Epoch 3: val_loss improved from 2.04567 to 1.87725, saving model to saved_models\audio_classification.hdf5
186/186 [==============================] - 0s 2ms/step - loss: 2.0055 - accuracy: 0.2990 - val_loss: 1.8772 - val_accuracy: 0.3464

Epoch 26/100
177/186 [===========================>..] - ETA: 0s - loss: 0.6221 - accuracy: 0.7860
Epoch 26: val_loss improved from 0.58137 to 0.54634, saving model to saved_models\audio_classification.hdf5
186/186 [==============================] - 0s 2ms/step - loss: 0.6290 - accuracy: 0.7829 - val_loss: 0.5463 - val_accuracy: 0.8187
Epoch 27/100
179/186 [===========================>..] - ETA: 0s - loss: 0.6015 - accuracy: 0.7947
Epoch 27: val_loss improved from 0.54634 to 0.53755, saving model to saved_models\audio_classification.hdf5
186/186 [==============================] - 0s 2ms/step - loss: 0.6074 - accuracy: 0.7932 - val_loss: 0.5376 - val_accuracy: 0.8139
Epoch 28/100
181/186 [============================>.] - ETA: 0s - loss: 0.5768 - accuracy: 0.8090
Epoch 28: val_loss improved from 0.53755 to 0.50349, saving model to saved_models\audio_classification.hdf5
186/186 [==============================] - 0s 2ms/step - loss: 0.5765 - accuracy: 0.8092 - val_loss: 0.5035 - val_accura

Epoch 53/100
184/186 [============================>.] - ETA: 0s - loss: 0.3878 - accuracy: 0.8653
Epoch 53: val_loss did not improve from 0.38663
186/186 [==============================] - 0s 2ms/step - loss: 0.3873 - accuracy: 0.8654 - val_loss: 0.4020 - val_accuracy: 0.8750
Epoch 54/100
173/186 [==========================>...] - ETA: 0s - loss: 0.3693 - accuracy: 0.8748
Epoch 54: val_loss did not improve from 0.38663
186/186 [==============================] - 0s 2ms/step - loss: 0.3690 - accuracy: 0.8749 - val_loss: 0.3968 - val_accuracy: 0.8769
Epoch 55/100
157/186 [========================>.....] - ETA: 0s - loss: 0.3687 - accuracy: 0.8680
Epoch 55: val_loss did not improve from 0.38663
186/186 [==============================] - 0s 2ms/step - loss: 0.3737 - accuracy: 0.8671 - val_loss: 0.4130 - val_accuracy: 0.8683
Epoch 56/100
176/186 [===========================>..] - ETA: 0s - loss: 0.3720 - accuracy: 0.8746
Epoch 56: val_loss did not improve from 0.38663
186/186 [==============

186/186 [==============================] - 0s 2ms/step - loss: 0.3164 - accuracy: 0.8927 - val_loss: 0.3625 - val_accuracy: 0.8912
Epoch 82/100
183/186 [============================>.] - ETA: 0s - loss: 0.3035 - accuracy: 0.8958
Epoch 82: val_loss did not improve from 0.36248
186/186 [==============================] - 0s 2ms/step - loss: 0.3043 - accuracy: 0.8954 - val_loss: 0.3821 - val_accuracy: 0.8941
Epoch 83/100
185/186 [============================>.] - ETA: 0s - loss: 0.3071 - accuracy: 0.8988
Epoch 83: val_loss did not improve from 0.36248
186/186 [==============================] - 0s 2ms/step - loss: 0.3078 - accuracy: 0.8984 - val_loss: 0.3690 - val_accuracy: 0.8931
Epoch 84/100
174/186 [===========================>..] - ETA: 0s - loss: 0.3045 - accuracy: 0.8955
Epoch 84: val_loss improved from 0.36248 to 0.35959, saving model to saved_models\audio_classification.hdf5
186/186 [==============================] - 0s 2ms/step - loss: 0.3126 - accuracy: 0.8942 - val_loss: 0.3596 -

In [24]:
y_pred=model.predict(X_test)

55/55 [==============================] - 0s 778us/step


In [26]:
score = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(X_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9738925099372864
Testing Accuracy:  0.8935317397117615
